# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
import pickle
from scipy import stats
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
import numpy as np
import pandas as pd
import warnings
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
from pypdf import PdfReader
from pdfminer.high_level import extract_text

# Load data

In [ ]:
path = 'E:/YandexDisk/Work/bbd/millenium'

feats_rows = {
    'ID': 3,
    'Дата и время обследования': 4,
    'Возраст': 5,
    'Пол': 5,
    'Рост': 6,
    'Вес': 6,
    'Окр. талии, см': 7,
    'Окр. бедер, см': 7,
    'Сопрот. акт. на 5 кГц, Ом': 12,
    'Сопрот. акт. на 50 кГц, Ом': 12,
    'Сопрот. реакт. на 50 кГц, Ом': 12,
    'Фазовый угол (50 кГц), град.': 13,
    'Клеточная жидкость, кг': 14,
    'Удельный основной обмен, ккал/м²/сут.': 15,
    'Индекс массы тела': 18,
    'Индекс массы тела (% от середины нормы)': 21,
    'Жировая масса, кг': 23,
    'Жировая масса, кг (% от середины нормы)': 26,
    'Тощая масса, кг': 28,
    'Тощая масса, кг (% от середины нормы)': 31,
    'Активная клеточная масса, кг': 34,
    'Активная клеточная масса, кг (% от середины нормы)': 37,
    'Доля активной клеточной массы, %': 40,
    'Доля активной клеточной массы, % (% от середины нормы)': 43,
    'Скелетно-мышечная масса, кг': 46,
    'Скелетно-мышечная масса, кг (% от середины нормы)': 49,
    'Доля скелетно-мышечной массы, %': 53,
    'Доля скелетно-мышечной массы, % (% от середины нормы)': 56,
    'Основной обмен, ккал/сут.': 59,
    'Основной обмен, ккал/сут. (% от середины нормы)': 62,
    'Общая жидкость, кг': 64,
    'Общая жидкость, кг (% от середины нормы)': 67,
    'Внеклеточная жидкость, кг': 70,
    'Внеклеточная жидкость, кг (% от середины нормы)': 73,
    'Индекс талия-бедра': 75,
    'Индекс талия-бедра (% от середины нормы)': 78,
    'Доля жировой массы, %': 81,
    'Доля жировой массы, % (% от середины нормы)': 84,
    
    'Отношение внеклеточной и клеточной жидкостей': 33,
    
    'Индекс жировой массы, кг/м²': 20,
    'Индекс жировой массы, кг/м² (% от середины нормы)': 23,
    'Индекс тощей массы, кг/м²': 24,
    'Индекс тощей массы, кг/м² (% от середины нормы)': 27,
    'Индекс активной клеточной массы, кг/м²': 28,
    'Индекс активной клеточной массы, кг/м² (% от середины нормы)': 31,
    'Индекс скелетно-мышечной клеточной массы, кг/м²': 32,
    'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)': 35,
    'Минеральная масса тела, кг': 38,
    'Минеральная масса тела, кг (% от середины нормы)': 41,
    'Минеральная масса мягких тканей, кг': 44,
    'Минеральная масса мягких тканей, кг (% от середины нормы)': 47,
    'Минеральная масса костной ткани, кг': 50,
    'Минеральная масса костной ткани, кг (% от середины нормы)': 53,
    'Доля минеральной массы в ТМ, %': 56,
    'Доля минеральной массы в ТМ, % (% от середины нормы)': 59,
    'Доля минеральной массы мягких тканей в ТМ, %': 63,
    'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)': 66,
    'Доля минеральной массы костной ткани в ТМ, %': 70,
    'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)': 73,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы': 78,
    'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)': 81
}


In [ ]:
fns = [
    'Бурминская Светлана Александровна, #2, 2024-10-11 10_55_46',
    'Нестерова Наталья Григорьевна, #1, 2024-10-03 11_08_44',
    'Гуржий Юлия Андреевна, #1, 2024-07-05 10_31_22',
    'Задоя  Ирина Петровна, #1, 2024-09-26 08_12_36',
    'Мустафина Диана Рашидовна, #1, 2025-02-21 10_53_39',
    'Вотина Полина Игоревна, #1, 2024-08-14 08_43_21'
]

df = pd.DataFrame(columns=list(feats_rows.keys()))

for fn in fns:
    print(fn)
    
    reader = PdfReader(f"{path}/{fn}.pdf")
    
    # Page 1
    page = reader.pages[0]
    lines = page.extract_text().splitlines()
    sample_id = lines[feats_rows['ID']]
    df.at[sample_id, 'ID'] = lines[feats_rows['ID']]
    date_time = re.findall(r"([0-9]{1,2}\.[0-9]{1,2}\.[0-9]{4})\s(\d{1,2}:\d{2}:\d{2})Дата\sобследования", lines[feats_rows['Дата и время обследования']])[0]
    df.at[sample_id, 'Дата и время обследования'] = f"{date_time[0]} {date_time[1]}"
    df.at[sample_id, 'Возраст'] = float(re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows['Возраст']])[0][0])
    df.at[sample_id, 'Пол'] = re.findall(r"(\d+), (.)Возраст, лет \/ Пол", lines[feats_rows['Пол']])[0][1]
    df.at[sample_id, 'Рост'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows['Рост']])[0][0])
    df.at[sample_id, 'Вес'] = float(re.findall(r"(\d+) \/ (\d+\.*\d*)Рост, см \/ Вес, кг", lines[feats_rows['Вес']])[0][1])
    df.at[sample_id, 'Окр. талии, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows['Окр. талии, см']])[0][0])
    df.at[sample_id, 'Окр. бедер, см'] = float(re.findall(r"(\d+) \/ (\d+)Окр. талии \/ Окр. бедер, см", lines[feats_rows['Окр. бедер, см']])[0][1])
    df.at[sample_id, 'Сопрот. акт. на 5 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows['Сопрот. акт. на 5 кГц, Ом']])[0][0])
    df.at[sample_id, 'Сопрот. акт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows['Сопрот. акт. на 50 кГц, Ом']])[0][1])
    df.at[sample_id, 'Сопрот. реакт. на 50 кГц, Ом'] = float(re.findall(r"(\d+) \/ (\d+) \/ (\d+)", lines[feats_rows['Сопрот. реакт. на 50 кГц, Ом']])[0][2])
    df.at[sample_id, 'Фазовый угол (50 кГц), град.'] = float(lines[feats_rows['Фазовый угол (50 кГц), град.']])
    df.at[sample_id, 'Клеточная жидкость, кг'] = float(lines[feats_rows['Клеточная жидкость, кг']])
    df.at[sample_id, 'Удельный основной обмен, ккал/м²/сут.'] = float(lines[feats_rows['Удельный основной обмен, ккал/м²/сут.']])
    df.at[sample_id, 'Индекс массы тела'] = float(lines[feats_rows['Индекс массы тела']])
    df.at[sample_id, 'Индекс массы тела (% от середины нормы)'] = float(lines[feats_rows['Индекс массы тела (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Жировая масса, кг'] = float(lines[feats_rows['Жировая масса, кг']])
    df.at[sample_id, 'Жировая масса, кг (% от середины нормы)'] = float(lines[feats_rows['Жировая масса, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Тощая масса, кг'] = float(lines[feats_rows['Тощая масса, кг']])
    df.at[sample_id, 'Тощая масса, кг (% от середины нормы)'] = float(lines[feats_rows['Тощая масса, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Активная клеточная масса, кг'] = float(lines[feats_rows['Активная клеточная масса, кг']])
    df.at[sample_id, 'Активная клеточная масса, кг (% от середины нормы)'] = float(lines[feats_rows['Активная клеточная масса, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля активной клеточной массы, %'] = float(lines[feats_rows['Доля активной клеточной массы, %']])
    df.at[sample_id, 'Доля активной клеточной массы, % (% от середины нормы)'] = float(lines[feats_rows['Доля активной клеточной массы, % (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Скелетно-мышечная масса, кг'] = float(lines[feats_rows['Скелетно-мышечная масса, кг']])
    df.at[sample_id, 'Скелетно-мышечная масса, кг (% от середины нормы)'] = float(lines[feats_rows['Скелетно-мышечная масса, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля скелетно-мышечной массы, %'] = float(lines[feats_rows['Доля скелетно-мышечной массы, %']])
    df.at[sample_id, 'Доля скелетно-мышечной массы, % (% от середины нормы)'] = float(lines[feats_rows['Доля скелетно-мышечной массы, % (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Основной обмен, ккал/сут.'] = float(lines[feats_rows['Основной обмен, ккал/сут.']])
    df.at[sample_id, 'Основной обмен, ккал/сут. (% от середины нормы)'] = float(lines[feats_rows['Основной обмен, ккал/сут. (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Общая жидкость, кг'] = float(lines[feats_rows['Общая жидкость, кг']])
    df.at[sample_id, 'Общая жидкость, кг (% от середины нормы)'] = float(lines[feats_rows['Общая жидкость, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Внеклеточная жидкость, кг'] = float(lines[feats_rows['Внеклеточная жидкость, кг']])
    df.at[sample_id, 'Внеклеточная жидкость, кг (% от середины нормы)'] = float(lines[feats_rows['Внеклеточная жидкость, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Индекс талия-бедра'] = float(lines[feats_rows['Индекс талия-бедра']])
    df.at[sample_id, 'Индекс талия-бедра (% от середины нормы)'] = float(lines[feats_rows['Индекс талия-бедра (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля жировой массы, %'] = float(lines[feats_rows['Доля жировой массы, %']])
    df.at[sample_id, 'Доля жировой массы, % (% от середины нормы)'] = float(lines[feats_rows['Доля жировой массы, % (% от середины нормы)']].replace('%', ''))

    # Page 3
    lines = reader.pages[2].extract_text().splitlines()
    df.at[sample_id, 'Отношение внеклеточной и клеточной жидкостей'] = float(re.findall(r"Ваш показатель ВКЖ\/КЖ составляет: (\d+\.*\d*) \(диапазон нормальных значений", lines[feats_rows['Отношение внеклеточной и клеточной жидкостей']])[0])

    # Page 5
    lines = reader.pages[4].extract_text().splitlines()
    df.at[sample_id, 'Индекс жировой массы, кг/м²'] = float(lines[feats_rows['Индекс жировой массы, кг/м²']])
    df.at[sample_id, 'Индекс жировой массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows['Индекс жировой массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Индекс тощей массы, кг/м²'] = float(lines[feats_rows['Индекс тощей массы, кг/м²']])
    df.at[sample_id, 'Индекс тощей массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows['Индекс тощей массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Индекс активной клеточной массы, кг/м²'] = float(lines[feats_rows['Индекс активной клеточной массы, кг/м²']])
    df.at[sample_id, 'Индекс активной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows['Индекс активной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м²'] = float(lines[feats_rows['Индекс скелетно-мышечной клеточной массы, кг/м²']])
    df.at[sample_id, 'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)'] = float(lines[feats_rows['Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Минеральная масса тела, кг'] = float(lines[feats_rows['Минеральная масса тела, кг']])
    df.at[sample_id, 'Минеральная масса тела, кг (% от середины нормы)'] = float(lines[feats_rows['Минеральная масса тела, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Минеральная масса мягких тканей, кг'] = float(lines[feats_rows['Минеральная масса мягких тканей, кг']])
    df.at[sample_id, 'Минеральная масса мягких тканей, кг (% от середины нормы)'] = float(lines[feats_rows['Минеральная масса мягких тканей, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Минеральная масса костной ткани, кг'] = float(lines[feats_rows['Минеральная масса костной ткани, кг']])
    df.at[sample_id, 'Минеральная масса костной ткани, кг (% от середины нормы)'] = float(lines[feats_rows['Минеральная масса костной ткани, кг (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля минеральной массы в ТМ, %'] = float(lines[feats_rows['Доля минеральной массы в ТМ, %']])
    df.at[sample_id, 'Доля минеральной массы в ТМ, % (% от середины нормы)'] = float(lines[feats_rows['Доля минеральной массы в ТМ, % (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, %'] = float(lines[feats_rows['Доля минеральной массы мягких тканей в ТМ, %']])
    df.at[sample_id, 'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)'] = float(lines[feats_rows['Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, %'] = float(lines[feats_rows['Доля минеральной массы костной ткани в ТМ, %']])
    df.at[sample_id, 'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)'] = float(lines[feats_rows['Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)']].replace('%', ''))
    df.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы'] = float(lines[feats_rows['Шкала оценки риска метаболического синдрома по проценту жировой массы']])
    df.at[sample_id, 'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)'] = float(lines[feats_rows['Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)']].replace('%', ''))

df = df.apply(pd.to_numeric, errors='ignore')
df['Дата и время обследования'] = pd.to_datetime(df['Дата и время обследования'], format='mixed')

In [ ]:
'Индекс жировой массы, кг/м²': 20,
'Индекс жировой массы, кг/м² (% от середины нормы)': 23,
'Индекс тощей массы, кг/м²': 24,
'Индекс тощей массы, кг/м² (% от середины нормы)': 27,
'Индекс активной клеточной массы, кг/м²': 28,
'Индекс активной клеточной массы, кг/м² (% от середины нормы)': 31,
'Индекс скелетно-мышечной клеточной массы, кг/м²': 32,
'Индекс скелетно-мышечной клеточной массы, кг/м² (% от середины нормы)': 35,
'Минеральная масса тела, кг': 38,
'Минеральная масса тела, кг (% от середины нормы)': 41,
'Минеральная масса мягких тканей, кг': 44,
'Минеральная масса мягких тканей, кг (% от середины нормы)': 47,
'Минеральная масса костной ткани, кг': 50,
'Минеральная масса костной ткани, кг (% от середины нормы)': 53,
'Доля минеральной массы в ТМ, %': 56,
'Доля минеральной массы в ТМ, % (% от середины нормы)': 59,
'Доля минеральной массы мягких тканей в ТМ, %': 63,
'Доля минеральной массы мягких тканей в ТМ, % (% от середины нормы)': 66,
'Доля минеральной массы костной ткани в ТМ, %': 70,
'Доля минеральной массы костной ткани в ТМ, % (% от середины нормы)': 73,
'Шкала оценки риска метаболического синдрома по проценту жировой массы': 78,
'Шкала оценки риска метаболического синдрома по проценту жировой массы (% от середины нормы)': 81

In [ ]:
# extract only text oriented up
print(page.extract_text(0))

# extract text oriented up and turned left
print(page.extract_text((0, 90)))

# extract text in a fixed width format that closely adheres to the rendered
# layout in the source pdf
print(page.extract_text(extraction_mode="layout"))

# extract text preserving horizontal positioning without excess vertical
# whitespace (removes blank and "whitespace only" lines)
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

# adjust horizontal spacing
print(page.extract_text(extraction_mode="layout", layout_mode_scale_weight=1.0))

# exclude (default) or include (as shown below) text rotated w.r.t. the page
print(page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False))

In [ ]:
reader = PdfReader(f"{path}/Бурминская Светлана Александровна, #2, 2024-10-11 10_55_46.pdf")
page = reader.pages[0]
print(page.extract_text(0))

In [ ]:
print(page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False))

In [ ]:
import re

def extract_floats(text):
    # Паттерн для поиска чисел с плавающей точкой в указанном формате
    pattern = r":\s*(\d+\.\d+)\s*кг"
    # Находим все совпадения в тексте
    matches = re.findall(pattern, text)
    # Конвертируем найденные строки в числа типа float
    return [float(match) for match in matches]

# Пример использования
text = """
Ваша активная клеточная масса составляет: 24.1 кг (диапазон)
Другие показатели: жировая масса 15.5 кг, вода: 30.0 кг.
Еще один пример: 42.8 кг (невероятный результат)!
"""

floats = extract_floats(text)
print("Найденные числа:", floats)

In [ ]:
import re

text = "39, ЖВозраст, лет / Пол"
result = re.findall(r"(\d+), (\S)Возраст.*", text)